In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image

import cv2 as cv

test_img_labels = pd.read_csv("/home/walke/college/cv/ass1/archive/test.csv")

new_test_data = [] 

for index in range(test_img_labels.shape[0]):
    img_csv = np.matrix(test_img_labels.iloc[index, 1:].values, dtype=np.uint8).reshape(28, 28)
    label = test_img_labels.iloc[index, 0]


    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True
 
    hog = cv.HOGDescriptor(winSize,blockSize,blockStride,
    cellSize,nbins,derivAperture,
    winSigma,histogramNormType,L2HysThreshold,
    gammaCorrection,nlevels, signedGradients)

    descriptor = hog.compute(img_csv)

    row = np.concatenate(([label], descriptor))
    new_test_data.append(row)

column_names = ["label"] + [f"feature_{i+1}" for i in range(len(new_test_data[0]) - 1)]
new_df = pd.DataFrame(new_test_data, columns=column_names)


new_df.to_csv("hog_test.csv", index=False)

train_img_labels = pd.read_csv("/home/walke/college/cv/ass1/archive/train.csv")

new_train_data = [] 

for index in range(train_img_labels.shape[0]):
    img_csv = np.matrix(train_img_labels.iloc[index, 1:].values, dtype=np.uint8).reshape(28, 28)
    label = train_img_labels.iloc[index, 0]


    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True
 
    hog = cv.HOGDescriptor(winSize,blockSize,blockStride,
    cellSize,nbins,derivAperture,
    winSigma,histogramNormType,L2HysThreshold,
    gammaCorrection,nlevels, signedGradients)

    descriptor = hog.compute(img_csv)

    row = np.concatenate(([label], descriptor))
    new_train_data.append(row)

column_names = ["label"] + [f"feature_{i+1}" for i in range(len(new_train_data[0]) - 1)]
new_df = pd.DataFrame(new_train_data, columns=column_names)


new_df.to_csv("hog_train.csv", index=False)



In [2]:
class hogImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):

        img_csv = self.img_labels.iloc[idx, 1:].values
        image = img_csv.reshape(9, 9)

        label = self.img_labels.iloc[idx, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(int(label))
        return image, label
    

training_data_hog = hogImageDataset(
    annotations_file = "hog_train.csv",
    transform=ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

test_data_hog = hogImageDataset(
    annotations_file = "hog_test.csv",
    transform=ToTensor(),
    target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
)

train_dataset_hog, val_dataset_hog = torch.utils.data.random_split(training_data_hog, [int(0.75*len(training_data_hog)), int(0.25*len(training_data_hog))])


In [3]:
from torch.utils.data import DataLoader
train_dataloader_hog = DataLoader(train_dataset_hog, batch_size=64, shuffle=True)
val_dataloader_hog = DataLoader(val_dataset_hog, batch_size=64, shuffle=True)
test_dataloader_hog = DataLoader(test_data_hog, batch_size=64, shuffle=True)

In [4]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
class NeuralNetwork_hog(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(9*9, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

model_hog = NeuralNetwork_hog().to(device)
print(model_hog)

for name, param in model_hog.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")



NeuralNetwork_hog(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=81, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 81]) | Values : tensor([[-0.0948, -0.0733, -0.0331,  0.0852,  0.0335,  0.0813,  0.0750, -0.1038,
         -0.0309,  0.0683,  0.0234,  0.0170,  0.0881,  0.1013, -0.0531,  0.0334,
          0.0232, -0.0358, -0.0420, -0.0741, -0.0854, -0.0515,  0.0475, -0.0406,
         -0.0939, -0.0795,  0.0765,  0.0942, -0.0012,  0.0535,  0.1036,  0.0182,
          0.0618, -0.0425,  0.0455, -0.0630,  0.1044,  0.0867,  0.0190,  0.0766,
         -0.0929, -0.0346, -0.0534, -0.1029, -0.0136,  0.0650,  0.0935, -0.0957,
          0.0077,  0.0930,  0.0537,  0.0047,  0.0196,  0.0073, -0.1109, -0.0932,
          0.0592, -0.0081,  0.0577, 

In [6]:
batch_size=64

from tqdm import tqdm

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    total = len(dataloader)

    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in tqdm(enumerate(dataloader),desc="train", total = total):
        # Compute prediction and loss
        X, y = X.to(device).float(), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



def test_loop(dataloader, model, loss_fn, model_path):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.load_state_dict(torch.load(model_path))

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


def val_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            
            y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    val_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Validation loss: {val_loss:>8f} \n")

    return correct

In [7]:
learning_rate_hog = 1e-3
batch_size_hog = 64
epochs_hog = 20

loss_fn_hog = nn.CrossEntropyLoss()
optimizer_hog = torch.optim.SGD(model_hog.parameters(), lr=learning_rate_hog)

# model.to(device)
val_accuracy_highest_hog = 0
val_accuracy_curr_hog = 0

for t in range(epochs_hog):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader_hog, model_hog, loss_fn_hog, optimizer_hog)
    val_accuracy_curr_hog = val_loop(test_dataloader_hog, model_hog, loss_fn_hog)
    if val_accuracy_curr_hog > val_accuracy_highest_hog:
        val_accuracy_highest_hog = val_accuracy_curr_hog
        model_path = 'model_1_hog'
        print(f"Saving Model at epoch: {t+1}\n")
        torch.save(model_hog.state_dict(), model_path)

    
print("Done!")

test_loop(test_dataloader_hog, model_hog, loss_fn_hog, "model_1_hog")

Epoch 1
-------------------------------


train:   0%|          | 0/704 [00:00<?, ?it/s]

train: 100%|██████████| 704/704 [00:09<00:00, 73.51it/s]


Validation Error: 
 Accuracy: 20.2%, Avg Validation loss: 2.284126 

Saving Model at epoch: 1

Epoch 2
-------------------------------


train: 100%|██████████| 704/704 [00:11<00:00, 62.97it/s]


...

Validation Error: 
 Accuracy: 65.7%, Avg Validation loss: 1.193297 

Saving Model at epoch: 19

Epoch 20
-------------------------------


train: 100%|██████████| 704/704 [00:09<00:00, 74.80it/s]


Validation Error: 
 Accuracy: 66.4%, Avg Validation loss: 1.158844 

Saving Model at epoch: 20

Done!


/tmp/ipykernel_90995/128364945.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Test Error: 
 Accuracy: 66.4%, Avg loss: 1.158937 

